### Your lab resolution :

In [112]:
import requests
import pandas as pd
from pandas.io.json import json_normalize

#For obvious reasons, being this an open file, I take away my login data.
#I commented the results in order to share the file without running it.
username = 
token = 

github = requests.get("https://api.github.com/users/ta-data-pt-rmt/repos?per_page=100", auth=(username,token))
github

<Response [200]>

In [ ]:
#Challenge 1

In [113]:
repos = github.json()
#repos

In [114]:
##Results for all forked projects from TA-DATA-PT-RMT

Languages = []
count = 0
count2 = 0
for item in repos:
    url2 = item["forks_url"]
    forks = requests.get(url2, auth=(username,token))
    forks2 = forks.json()
    #print(item["name"])
    count2 +=1
    for element in forks2:
        #print(element["language"])
        count +=1
        if element["language"] not in Languages:
            Languages.append(element["language"])

print("Total forks: "+str(count))
print("Total projects: "+str(count2))
print("Used Languages: ")
print(*Languages, sep = ", ")

#Total forks: 435
#Total projects: 50
#Used Languages: 
#None, Jupyter Notebook, Python

Total forks: 435
Total projects: 50
Used Languages: 
None, Jupyter Notebook, Python


In [142]:
#Same as before, but just using my own repositories. No forks here...

My_github = requests.get("https://api.github.com/users/jadcarpan/repos?per_page=100", auth=(username,token))
My_repos = My_github.json()
My_languages = []
count3 = 0
for item in My_repos:
    count3 +=1
    if item["language"] not in My_languages:
        My_languages.append(item["language"])
        
print("Total projects: "+str(count3))
print("Used Languages: ")
print(*My_languages,sep = ", ")


#Total projects: 21
#Used Languages: 
#None, Jupyter Notebook, Python


Total projects: 21
Used Languages: 
None, Jupyter Notebook, Python


In [ ]:
#Challenge 2

In [141]:
web = requests.get("https://api.github.com/users/ta-data-pt-rmt/repos", auth=(username,token))
web

<Response [200]>

In [172]:
web_commits = web.json()
web_commits
labs = []
for item in web_commits:
    labs.append(item["commits_url"])
labs
#This way I can select the right path to the right repo and continue with it below

#To get some results, I tracked the activity up to the beginning of the Bootcamp instead of using just 2 weeks (0 commits)
commits1 = requests.get("https://api.github.com/repos/ta-data-pt-rmt/lab-mysql-first-queries/commits?since=2022-02-22T00:00:01Z", auth=(username,token))
commit = commits1.json()
commit
number = 0
for item in commit:
    number += 1
    
print(number)
#result = 3

3
